In [4]:
features_df = pd.read_csv("../../data/graph-save/node_features.csv")
len(features_df)

154

In [5]:
import pandas as pd

SAVE_DIR = "../../data/graph-save"

df = pd.read_csv("../../data/clean_data.csv")
features_df = pd.read_csv("../../data/graph-save/node_features.csv")
links_df = pd.read_csv("../../data/links_graph.csv")

# display(df)
# display(features_df)
# display(links_df)

new_features_df = features_df[['node_id', 'churn_rate', 'sessions_count']].copy()
additional_columns = df[['node_id', 'screen', 'feature', 'action']].copy()
additional_columns = additional_columns.drop_duplicates(subset="node_id")
new_features_df = pd.merge(new_features_df, additional_columns, on='node_id', how='left')

# Очищаем память от исходного df
del df
del additional_columns

# Отображаем результат
display(new_features_df)
display(links_df)

# 'Открытие экрана'

,node_id,churn_rate,sessions_count,screen,feature,action
0,0072f89b60d46ef6f2094949d8831f13,0.087339,74914,Важное,Просмотр уведомления,Тап на уведомление
1,02b207cc24a78c1942161bafc72fe532,0.080450,71489,Еще,Переход в раздел 'Опросы и собрания собственни...,Тап на кнопку 'Опросы и собрания собственников'
2,05084b69eadb4959371691236de248f0,0.000000,6,Новое ОСС,Удаление вопроса,Тап на кнопку 'Удалить'
3,05aa62cfe2beb31d4ecc652cddec5689,0.000000,12,Объявления,Редактирование опубликованного объявления,Тап на кнопку 'Редактировать'
4,061da77ad7d449a342174810fbf72350,0.000000,6,Гостевой доступ,Раскрытие вкладки 'Архив',Тап на кнопку 'Архив'
...,...,...,...,...,...,...
149,f2f9d242858a788cad0cd1e66264f25b,0.016351,2737,Услуги,Переход к городским сервисам,Тап на кнопку 'Городские сервисы'
150,f38e3fd9c83a13ec4cc1da34306c886d,0.000000,122,Новое ОСС,Переход к определению даты начала и продолжите...,Тап на кнопку 'Далее'
151,f41232f99b7092c102621d06b65d813d,0.058824,4,Гостевой доступ,Переход в справку по функционалу доступов,Тап на кнопку 'Инфо'
152,f45c33fe096f325179397402f5b9eb00,0.048016,7530,Услуги,Открытие экрана,Открытие экрана


,source_id,target_id
0,875e50d28d26088a760866c1125ff124,5281fb229131fa372bd15589fed81bc7
1,5281fb229131fa372bd15589fed81bc7,ac398d9f0ba05d231b9d183feb595b12
2,ac398d9f0ba05d231b9d183feb595b12,875e50d28d26088a760866c1125ff124
3,875e50d28d26088a760866c1125ff124,c3617d19147db0f2c720ae56324619a3
4,875e50d28d26088a760866c1125ff124,5d25ba67e2bc07adddc59f7a0e0baacd
...,...,...
3024,e9c334108c17d0d6014b73f196b775f4,996468d1c41acb5b3e3269db1fe084b1
3025,996468d1c41acb5b3e3269db1fe084b1,e9c334108c17d0d6014b73f196b775f4
3026,b0af5b926c05544c79e497f6b54fe815,172ee4dedca1786e2e6d90ebd4812463
3027,172ee4dedca1786e2e6d90ebd4812463,ac398d9f0ba05d231b9d183feb595b12


## Формирование полного графа

### Nodes

- `name`
    - `feature`, если `feature` != "Открытие экрана"
    - иначе `screen`

- `symbolSize`
    - пропорционально `sessions_count`
    - нормализация по максимуму

- `color`
    - 6 порогов по `churn_rate`
    - от #5470c6 → красно-фиолетовый

- `sessions_count`, `churn_rate` — как есть

### Links

- `source`, `target` — из links_df
- `sessions_count` — берём у target-ноды
- `width` — масштабируем по `sessions_count`
- `color` — по `churn_rate` target-ноды

In [38]:
import os
import json
import numpy as np

# ---------------------------
# Настройки визуализации
# ---------------------------

MIN_NODE_SIZE = 20
MAX_NODE_SIZE = 80

MIN_EDGE_WIDTH = 1
MAX_EDGE_WIDTH = 5

CHURN_COLORS = [
    "#5470c6",  # 0
    "#6a5acd",
    "#8a2be2",
    "#a020f0",
    "#b0006d",
    "#8b0033",  # max
]


# ---------------------------
# Вспомогательные функции
# ---------------------------

def scale_value(value, min_val, max_val, out_min, out_max):
    if max_val == min_val:
        return (out_min + out_max) / 2
    return out_min + (value - min_val) / (max_val - min_val) * (out_max - out_min)


def churn_to_color(churn, max_churn, gamma=0.25):
    """
    Преобразуем churn_rate в индекс цвета с использованием степенной шкалы.
    gamma < 1 → малые значения быстрее становятся красными
    gamma = 1 → линейная шкала (как было)
    """
    if max_churn == 0:
        return CHURN_COLORS[0]

    # нормализация и степенная шкала
    normalized = (churn / max_churn) ** gamma

    idx = int(normalized * (len(CHURN_COLORS) - 1))
    idx = min(idx, len(CHURN_COLORS) - 1)
    return CHURN_COLORS[idx]


# ---------------------------
# Подготовка данных
# ---------------------------

node_df = new_features_df.copy()

node_df["name"] = np.where(
    node_df["feature"] != "Открытие экрана",
    node_df["feature"],
    node_df["screen"],
)

max_sessions = node_df["sessions_count"].max()
max_churn = node_df["churn_rate"].max()

# Быстрый доступ по node_id
node_map = {}

nodes = []
for _, row in node_df.iterrows():
    symbol_size = scale_value(
        row["sessions_count"],
        0,
        max_sessions,
        MIN_NODE_SIZE,
        MAX_NODE_SIZE,
    )

    color = churn_to_color(row["churn_rate"], max_churn)

    node = {
        "id": row["node_id"],
        "name": row["name"],
        "x": 0,
        "y": 0,
        "symbolSize": symbol_size,
        "sessions_count": int(row["sessions_count"]),
        "churn_rate": float(row["churn_rate"]),
        "itemStyle": {
            "color": color
        }
    }

    nodes.append(node)
    node_map[row["node_id"]] = node


# ---------------------------
# Links
# ---------------------------

links = []

# уникальные связи
links_df_unique = links_df[["source_id", "target_id"]].drop_duplicates()

# sessions_count по node_id
# sessions_map = (
#     new_features_df
#     .set_index("node_id")["sessions_count"]
#     .to_dict()
# )

# # фильтрация связей
# MIN_SESSIONS = 10

# links_df_unique = links_df_unique[
#     links_df_unique["source_id"].map(sessions_map).fillna(0) >= MIN_SESSIONS
# ]

# links_df_unique = links_df_unique[
#     links_df_unique["target_id"].map(sessions_map).fillna(0) >= MIN_SESSIONS
# ]

print("links after unique:", len(links_df))
print("links after filter:", len(links_df_unique))


for _, row in links_df_unique.iterrows():
    source = row["source_id"]
    target = row["target_id"]

    if target not in node_map:
        continue

    target_node = node_map[target]

    sessions = target_node["sessions_count"]

    width = scale_value(
        sessions,
        0,
        max_sessions,
        MIN_EDGE_WIDTH,
        MAX_EDGE_WIDTH,
    )

    link = {
        "source": source,
        "target": target,
        "sessions_count": sessions,
        "lineStyle": {
            "width": width,
            "color": target_node["itemStyle"]["color"],
            "curveness": 0.1,
            "opacity": 0.8,
        }
    }

    links.append(link)


# ---------------------------
# Финальный граф
# ---------------------------

graph = {
    "nodes": nodes,
    "links": links,
}

with open(os.path.join(SAVE_DIR, "graph_full.json"), "w", encoding="utf-8") as f:
    json.dump(graph, f, ensure_ascii=False, indent=2)

print(f"Saved graph_full.json ({len(nodes)} nodes, {len(links)} links)")


links after unique: 3029
links after filter: 3029
Saved graph_full.json (154 nodes, 2883 links)


### Усечённый граф

In [42]:
TOP_VISITS_PCTL = 0.96      # верхние 30% по посещаемости
BOTTOM_VISITS_PCTL = 0.50  # нижние 30% по посещаемости
CHURN_PCTL = 0.95          # верхние 30% по churn

# TOP_VISITS_PCTL = 1
# BOTTOM_VISITS_PCTL = 0
# CHURN_PCTL = 0.999

import numpy as np

# --- собираем массивы для порогов ---
link_sessions = np.array([l["sessions_count"] for l in links])

# churn берём у target
link_churn = np.array([
    node_map[l["target"]]["churn_rate"]
    for l in links
])

# --- считаем пороги ---
visits_top_thr = np.quantile(link_sessions, TOP_VISITS_PCTL)
visits_bottom_thr = np.quantile(link_sessions, BOTTOM_VISITS_PCTL)
churn_thr = np.quantile(link_churn, CHURN_PCTL)

# --- фильтрация ---
filtered_links = []

for link in links:
    sessions = link["sessions_count"]
    churn = node_map[link["target"]]["churn_rate"]

    # ❌ нижний мусор
    if sessions < visits_bottom_thr:
        continue

    # ✅ основной поток
    if sessions >= visits_top_thr:
        filtered_links.append(link)
        continue

    # ✅ проблемные места
    if churn >= churn_thr and sessions >= visits_bottom_thr:
        filtered_links.append(link)
        continue

# --- собираем используемые ноды ---
used_node_ids = set()
for l in filtered_links:
    used_node_ids.add(l["source"])
    used_node_ids.add(l["target"])

filtered_nodes = [
    n for n in nodes
    if n["id"] in used_node_ids
]

graph_main = {
    "nodes": filtered_nodes,
    "links": filtered_links
}

with open(os.path.join(SAVE_DIR, "graph_main.json"), "w", encoding="utf-8") as f:
    json.dump(graph_main, f, ensure_ascii=False, indent=2)

print(
    f"graph_main.json: "
    f"{len(filtered_nodes)} nodes, "
    f"{len(filtered_links)} links"
)


graph_main.json: 106 nodes, 173 links
